### 梯度下降法的向量化

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
boston = datasets.load_boston()

In [3]:
X = boston.data
y = boston.target

In [4]:
X = X[y<50.0]
y = y[y<50.0]

In [5]:
from playML.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,seed = 666)  # 独立的

In [6]:
from playML.LinearRegression import LinearRegression
lin_reg1 = LinearRegression()
%time lin_reg1.fit_normal(X_train,y_train)   # 正规方程解 
lin_reg1.score(X_test,y_test)

Wall time: 99.7 ms


0.8129802602658359

### 梯度下降法

In [8]:
lin_reg2 = LinearRegression()  # 
lin_reg2.fit_gd(X_train,y_train)  #  overflow encountered in reduce

C:\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
H:\jupyter\ml\6_梯度下降法\playML\LinearRegression.py:32: RuntimeWarning: overflow encountered in square
  return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
H:\jupyter\ml\6_梯度下降法\playML\LinearRegression.py:53: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):


LinearRegression()

In [9]:
lin_reg2.coef_  # 系数  溢出

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [10]:
X_train[:10] # 

array([[1.42362e+01, 0.00000e+00, 1.81000e+01, 0.00000e+00, 6.93000e-01,
        6.34300e+00, 1.00000e+02, 1.57410e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.96900e+02, 2.03200e+01],
       [3.67822e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 7.70000e-01,
        5.36200e+00, 9.62000e+01, 2.10360e+00, 2.40000e+01, 6.66000e+02,
        2.02000e+01, 3.80790e+02, 1.01900e+01],
       [1.04690e-01, 4.00000e+01, 6.41000e+00, 1.00000e+00, 4.47000e-01,
        7.26700e+00, 4.90000e+01, 4.78720e+00, 4.00000e+00, 2.54000e+02,
        1.76000e+01, 3.89250e+02, 6.05000e+00],
       [1.15172e+00, 0.00000e+00, 8.14000e+00, 0.00000e+00, 5.38000e-01,
        5.70100e+00, 9.50000e+01, 3.78720e+00, 4.00000e+00, 3.07000e+02,
        2.10000e+01, 3.58770e+02, 1.83500e+01],
       [6.58800e-02, 0.00000e+00, 2.46000e+00, 0.00000e+00, 4.88000e-01,
        7.76500e+00, 8.33000e+01, 2.74100e+00, 3.00000e+00, 1.93000e+02,
        1.78000e+01, 3.95560e+02, 7.56000e+00],
       [2.49800e-02, 0.00000e+

In [11]:
lin_reg2.fit_gd(X_train,y_train,eta = 0.000001)

LinearRegression()

In [12]:
lin_reg2.score(X_test,y_test)  # 步长太小，n_iters=1e4 不够，应该扩大循环次数

0.2755663485338923

In [13]:
%time lin_reg2.fit_gd(X_train,y_train,eta = 0.000001,n_iters = 1e6)

Wall time: 44.8 s


LinearRegression()

In [15]:
lin_reg2.score(X_test,y_test)  # 尽管循环次数很多，依旧没有达到最小值，因为要处理的数据不再一个规模上 所以要进行数归一化

0.7541852353980764

### 使用梯度下降法前进行数据归一化

In [14]:
from sklearn.preprocessing import StandardScaler   # 均值方差归一化

In [15]:
standardScaler = StandardScaler()
standardScaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [16]:
X_train_standard = standardScaler.transform(X_train)

In [17]:
X_train_standard.shape

(392, 13)

In [18]:
X_train_standard[10:]

array([[ 1.09317061, -0.47293831,  1.0044968 , ...,  0.77931907,
         0.43942982,  0.23453068],
       [-0.3438348 ,  0.40351515, -1.07103108, ..., -2.62713925,
         0.38474026, -0.8634381 ],
       [ 0.25410812, -0.47293831,  1.0044968 , ...,  0.77931907,
         0.43942982, -0.32702587],
       ...,
       [-0.34208171, -0.47293831,  1.0044968 , ...,  0.77931907,
        -3.4542028 ,  1.52247879],
       [ 0.69055174, -0.47293831,  1.0044968 , ...,  0.77931907,
         0.210658  ,  1.00981398],
       [-0.39597267,  0.4911605 , -0.7934127 , ...,  0.25888794,
         0.17082375, -0.08117027]])

In [19]:
lin_reg3 = LinearRegression()
%time lin_reg3.fit_gd(X_train_standard,y_train)

Wall time: 223 ms


LinearRegression()

In [20]:
X_test_standard = standardScaler.transform(X_test) # 将测试数据归一化

In [21]:
lin_reg3.score(X_test_standard,y_test)  # 说明找到了损失函数的最小值

0.8129880620122235

### 梯度下降的优势

In [30]:
m = 1000
n = 5000
big_x = np.random.normal(size = (m,n))  # 矩阵  特征数是n
true_theta = np.random.uniform(0.0,100.0,size = n+1) # 前包后不包，float  
big_y = big_x.dot(true_theta[1:])+true_theta[0]+np.random.normal(0.,10.,size=m)  # 

In [31]:
true_theta

array([90.63889593,  3.18873553, 74.05353318, ...,  9.75091233,
       74.50747488,  8.48854603])

In [32]:
big_reg1 = LinearRegression()  # 耗时
%time big_reg1.fit_normal(big_x,big_y)

Wall time: 5 s


LinearRegression()

In [34]:
big_reg2 = LinearRegression()  #   加大特征数的话效果更明显
%time big_reg2.fit_gd(big_x,big_y)

Wall time: 4.6 s


LinearRegression()

In [27]:
a = np.random.uniform()
a

0.8646343674921918